# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [15]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Loaded dataset with 25000 training samples, 25000 test samples


In [16]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 11, 4, 86, 891, 234, 175, 4896, 614, 16, 2, 46, 34, 4, 2, 564, 5, 167, 50, 16, 4, 2, 4101, 2, 19, 4, 3810, 223, 113, 4, 4103, 3810, 2, 2089, 1679, 256, 34, 13, 70, 516, 101, 248, 2901, 2, 4, 3810, 336, 7, 6, 2487, 1440, 5, 4, 2, 2928, 2, 3187, 2408, 10, 10, 363, 131, 47, 57, 1120, 7, 4, 2, 7, 2, 4, 2, 1707, 16, 6, 2, 5, 12, 16, 669, 1266, 23, 4, 1565, 40, 6, 2, 37, 872, 8, 193, 6, 2, 4, 86, 840, 7, 4, 2, 16, 2, 4, 2, 320, 7, 4, 1707, 16, 2, 5, 4, 2, 1845, 1177, 57, 2, 42, 2, 91, 7, 4, 84, 144, 28, 2529, 245, 11, 641, 7, 89, 111, 84, 4674, 15, 1707, 15, 188, 2, 11, 2, 103, 12, 416, 94, 2, 91, 7, 14, 248, 1707, 286, 60, 3865, 10, 10, 12, 1578, 72, 7, 4, 136, 11, 945, 1147, 31, 121, 4, 2, 1936, 4, 1093, 5, 95, 2, 367, 40, 45, 93, 7, 2, 3803, 4587, 10, 10, 4, 123, 16, 38, 78, 12, 16, 1167, 8, 106, 60, 61, 322, 37, 9, 53, 2, 74, 13, 16, 2, 23, 1759, 1508, 51, 69, 72, 2, 16, 89, 14, 2, 100, 126, 79, 93, 26, 4, 1180, 7, 134, 183, 38, 343, 8, 1807, 15, 36, 191, 60, 2537, 98,

Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [17]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

--- Review (with words) ---
['the', 'this', 'of', 'how', 'unless', 'since', 'us', 'mentions', 'shown', 'with', 'and', 'some', 'who', 'of', 'and', 'violence', 'to', 'going', 'more', 'with', 'of', 'and', 'pays', 'and', 'film', 'of', 'unbelievably', 'whole', 'acting', 'of', 'stiller', 'unbelievably', 'and', 'desert', 'sight', 'anyone', 'who', 'was', 'well', 'car', 'think', 'day', 'regard', 'and', 'of', 'unbelievably', 'help', 'br', 'is', 'experiences', 'changes', 'to', 'of', 'and', 'ned', 'and', 'tune', 'unfortunate', 'i', 'i', 'until', 'these', 'there', 'even', 'positive', 'br', 'of', 'and', 'br', 'and', 'of', 'and', 'warning', 'with', 'is', 'and', 'to', 'that', 'with', 'episodes', 'week', 'are', 'of', 'provides', 'just', 'is', 'and', 'like', 'subject', 'in', 'long', 'is', 'and', 'of', 'how', 'particular', 'br', 'of', 'and', 'with', 'and', 'of', 'and', 'star', 'br', 'of', 'warning', 'with', 'and', 'to', 'of', 'and', 'taylor', 'producers', 'even', 'and', "it's", 'and', 'its', 'br', 'of', 

In [18]:
def FindMaxLength(lst): 
   
    maxLength = max(map(len, lst)) 
      
    return maxLength 
  

lst = X_train
print(FindMaxLength(lst)) 

2494


In [19]:
def FindMinLength(lst): 
   
    minLength = min(map(len, lst)) 
      
    return minLength 
  

lst = X_train
print(FindMinLength(lst)) 

11


Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

Maximum review length = 2494 words and Minimum review length = 11 words.


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [20]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train = sequence.pad_sequences(X_train,maxlen=max_words)
print(X_train.shape)
X_test = sequence.pad_sequences(X_test,maxlen=max_words)
print(X_test.shape)

(25000, 500)
(25000, 500)


### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [21]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

Trainable parameters = 213,301

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [22]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [23]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 3

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]  # first batch_size samples
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]  # rest for training

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)
model.fit(X_train2, y_train2,
          validation_data=(X_valid, y_valid),
          batch_size=batch_size, epochs=num_epochs)
# TODO: Train your model


Train on 24936 samples, validate on 64 samples
Epoch 1/3
24936/24936 [==============================] - 520s 21ms/step - loss: 0.4593 - acc: 0.7777 - val_loss: 0.4282 - val_acc: 0.8281
Epoch 2/3
24936/24936 [==============================] - 520s 21ms/step - loss: 0.2974 - acc: 0.8805 - val_loss: 0.4466 - val_acc: 0.8281
Epoch 3/3
24936/24936 [==============================] - 519s 21ms/step - loss: 0.2567 - acc: 0.8995 - val_loss: 0.4199 - val_acc: 0.8594


In [26]:
# Save your model, so that you can quickly load it in future (and perhaps resume training)
import os
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join(cache_dir, model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

NameError: name 'cache_dir' is not defined

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [27]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.87316


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

...

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!